In [7]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import json
from PIL import Image
from urllib.parse import urlparse
import urllib.request
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from msedge.selenium_tools import Edge, EdgeOptions

In [18]:
driver = webdriver.Chrome(executable_path="C:\webDriver\chromedriver.exe")

C:\Users\nurul\AppData\Local\Temp/ipykernel_17696/3779701882.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\webDriver\chromedriver.exe")


In [15]:
def get_url(search_text):
    "generate url from Search term"
    template = 'https://www.amazon.com.tr/s?k={}&__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'
    search_text = search_text.replace(' ', '+')
    return template.format(search_text)


In [17]:
url = get_url('bilgisayar')
url

'https://www.amazon.com.tr/s?k=bilgisayar&__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'

In [19]:
driver.get(url)

In [54]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
results = soup.find_all('div',{'data-component-type':'s-search-result'})
data = []

for item in results:
    record = extact_record(item)
    if record:
        data.append(record)
        
        
data[1]

['LenovoYoga7DizüstüBilgisayar,14"FHD,IntelCorei7-1165G7,8GBRAM,512GBSSD,82BH00AGTX,Windows10',
 '12.199,00TL',
 '',
 '',
 'https://www.amazon.com.tr//Lenovo-Diz%C3%BCst%C3%BC-Bilgisayar-i7-1165G7-82BH00AGTX/dp/B095SWT5GM/ref=sr_1_2?__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&keywords=bilgisayar&qid=1636824140&sr=8-2']

### Generalize the Pattern

In [53]:
def extact_record(item):
    
    atag = item.h2.a
    
    try:
        description = re.sub('\s+', '',atag.text)
    except Exception:
        description = ""
        
    try:    
        url= re.sub('\s+', '','https://www.amazon.com.tr/' + atag['href'])
    except Exception:
        url =""
        
    try:    
        price = re.sub('\s+', '',item.find('span', 'a-price').find('span','a-offscreen').text.replace('\xa0',' '))
    except Exception:
        price=""
        
    try:    
        rating = re.sub('\s+', '',item.i.text)
    except Exception:
        rating =""
        
    try:     
        review_count=re.sub('\s+', '',item.find('span',{"class":"a-size-base"}).text)
    except Exception:
        review_count =""
    
    
    result = [description,price,rating,review_count,url]
    
    return result

# Getting The Next Page

In [55]:
def get_url(search_text):
    "generate url from Search term"
    template = 'https://www.amazon.com.tr/s?k={}&__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_2'
    
    search_text = search_text.replace(' ', '+')
    
    url = template.format(search_text)
    
    url += '&page{}'
    
    return url


# Pull All Together

In [75]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
import json
from PIL import Image
from urllib.parse import urlparse
import urllib.request
import time

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from msedge.selenium_tools import Edge, EdgeOptions

def extract_record(item):
    
    atag = item.h2.a
    
    try:
        description = re.sub('\s+', '',atag.text)
    except Exception:
        description = ""
        
    try:    
        url= re.sub('\s+', '','https://www.amazon.com.tr/' + atag['href'])
    except Exception:
        url =""
        
    try:    
        price = re.sub('\s+', '',item.find('span', 'a-price').find('span','a-offscreen').text.replace('\xa0',' '))
    except Exception:
        price=""
        
    try:    
        rating = re.sub('\s+', '',item.i.text)
    except Exception:
        rating =""
        
    try:     
        review_count=re.sub('\s+', '',item.find('span',{"class":"a-size-base"}).text)
    except Exception:
        review_count =""
    
    
    result = [description,price,rating,review_count,url]
    
    return result

def get_url(search_text):
    "generate url from Search term"
    search_text = search_text.replace(' ', '+')
    
    template = 'https://www.amazon.com.tr/s?k=:search&page=:pageNumber&__mk_tr_TR=%C3%85M%C3%85%C5%BD%C3%95%C3%91&qid=1636827846&ref=sr_pg_2'
    
    url = template.replace(':search', search_text)
        
    return url


def main(search_term):
    #run main routine
    driver = webdriver.Chrome(executable_path="C:\webDriver\chromedriver.exe")
    
    records = []
    
    url = get_url(search_term)
    
    
    for page in range(1,21):
        driver.get(url.replace(':pageNumber', str(page)))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div',{'data-component-type':'s-search-result'})
        
        
        
        for item in results:
            record = extract_record(item)
            
            if record:
                records.append(record)
                
                
                
    driver.close()
    #Save 
    df = pd.DataFrame(data, columns = ['Description', 'price', 'rating', 'review_count', 'url'])
    df.to_excel ('C:/Users/nurul/OneDrive/Masaüstü/New folder/export_dataframe2.xlsx', index = False, header=True)


In [76]:
main('bilgisayar')

C:\Users\nurul\AppData\Local\Temp/ipykernel_17696/4226763522.py:66: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:\webDriver\chromedriver.exe")
